In [ ]:
%load_ext dotenv
%dotenv

In [11]:

from autogen import ConversableAgent, AssistantAgent, GroupChat, GroupChatManager
from autogen.coding import DockerCommandLineCodeExecutor
from pathlib import Path
import os

llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]}

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)
executor = DockerCommandLineCodeExecutor(work_dir=work_dir, image="glam-cli", stop_container=True, timeout=30)

glam_tx_executor = ConversableAgent(
    name="glam_tx_executor",
    llm_config=False,
    code_execution_config={
        "executor": executor,
        "last_n_messages": 2,
    },
    human_input_mode="NEVER",
)

glam_agent_system_prompt = """
You have been given coding capability to solve tasks using bash script. The command-line tool you use can be invoked
by `node /mnt/glam/dist/cli/main.js`.

Commands available:
1. env
    - description: print environment setup
    - example: `node /mnt/glam/dist/cli/main.js env`
2. fund balances
    - description: display token balances of the fund
    - example: `node /mnt/glam/dist/cli/main.js fund balances`
3. fund swap [-s <slippageBps>] [-m <maxAccounts][-d] <from_mint> <to_mint> <amount>
    - description: swap tokens, <from_mint> and <to_mint> are token mint addresses; slippageBps is optional but recommended to set to 50; -d is optional but recommended to set; -m is optional but recommended to set to 20
    - example: `node /mnt/glam/dist/cli/main.js fund swap So11111111111111111111111111111111111111112 EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v 0.1`

Always ignore "bigint: Failed to load bindings, pure JS will be used (try npm run rebuild?)" warning.
"""

#
# Customize the system prompt
#
strategy_agent_system_prompt = """
You're a crypto fund manager that manages an index fund on the Solana blockchain that tracks 50% USDC and 50% WSOL.

Here are some tokens and their mint addresses, you need them when you want to swap tokens for rebalancing:
- WSOL: So11111111111111111111111111111111111111112
- USDC: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v

You should check fund balances and swap between USDC and WSOL if needed to make the fund 50% USDC and 50% WSOL. If the difference between the two token's values is less than 5 USD, then do nothing.

For simplicity, you can assume that SOL and WSOL are interchangeable names of the same token.
"""

glam_agent = AssistantAgent(name="glam_agent", llm_config=llm_config, system_message=glam_agent_system_prompt)
strategy_agent = AssistantAgent(name="strategy_agent", llm_config=llm_config, system_message=strategy_agent_system_prompt)

groupchat = GroupChat(agents=[glam_tx_executor, glam_agent, strategy_agent], messages=[], max_round=15)
groupchat_manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
import pprint

chat_result = glam_agent.initiate_chat(
    groupchat_manager, message="""Hi there, I'm your helpful glam agent."""
)
pprint.pp(chat_result)